# Batch Segmentation for Remote Sensing Imagery with SAM 3

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/sam3_batch_segmentation.ipynb)

This notebook demonstrates how to do batch segmentation for remote sensing imagery with SAM 3.

## Installation

First, make sure you have the required dependencies installed:

In [ ]:
# %pip install "segment-geospatial[samgeo3]"

## Import Libraries


In [ ]:
import leafmap
from samgeo import SamGeo3, download_file

## Download Sample Data

Let's download sample satellite images covering the University of California, Berkeley, for testing:


In [ ]:
image_paths = []
for i in range(1, 5):
    url = f"https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley_{i}.tif"
    image_path = download_file(url)
    image_paths.append(image_path)

In [ ]:
m = leafmap.Map()
for i, image_path in enumerate(image_paths):
    m.add_raster(image_path, layer_name=f"image_{i + 1}")
m

## Request access to SAM3

To use SAM3, you need to request access by filling out this form on Hugging Face: https://huggingface.co/facebook/sam3

Once you have access, uncomment the following code block and run it.

In [ ]:
# from huggingface_hub import login
# login()

## Initialize SAM3

When initializing SAM3, you can choose the backend from "meta", or "transformers".

In [ ]:
sam3 = SamGeo3(backend="meta", device=None, checkpoint_path=None, load_from_HF=True)

## Set the image batch

In [ ]:
sam3.set_image_batch(image_paths)

## Generate masks with text prompt

Generate masks for all images with a text prompt

In [ ]:
sam3.generate_masks_batch("building", min_size=100)

In [ ]:
# Access results for each image
for i, result in enumerate(sam3.batch_results):
    print(f"Image {i + 1}: Found {len(result['masks'])} objects")
    # result contains: masks, boxes, scores, image, source

## Show results

In [ ]:
# Visualize all annotations in a grid
sam3.show_anns_batch(ncols=2, show_bbox=True, show_score=True)

![](https://github.com/user-attachments/assets/0c4f0b4b-4104-4a43-9173-ab76e0d2d71a)

## Save results

In [ ]:
sam3.show_anns_batch(output_dir="output/annotations/", prefix="ann", dpi=300)

In [ ]:
# Save all masks to disk
saved_files = sam3.save_masks_batch(
    output_dir="output/", prefix="building_mask", unique=True
)